In [46]:
import pandas as pd

data = pd.read_parquet("../data/dataset/shs100k2/data.parquet")
data["performer_single"] = data.performer.apply(lambda x: x[0] if len(x) > 0 else '')
cols = ['set_id', 'yt_id', 'title', 'performer', 'performer_single', 'viewcount', 'split', 'TEXT', 'IOB']
data = data[cols]

# get and preprocess wikidata
cols = ["link", "name", "sitelinks"]
data_groups = pd.read_json("../data/wikidata/groups.json")
data_groups.musicalGroupLabel = data_groups.musicalGroupLabel.str.lower()
data_groups.columns = cols
data_groups["type"] = "Group"

data_humans1 = pd.read_json("../data/wikidata/humans1.json")
data_humans1.personLabel = data_humans1.personLabel.str.lower()
data_humans1.columns = cols
data_humans1["type"] = "Person"

data_wikidata = pd.concat([data_groups, data_humans1], axis=0, ignore_index=True).drop_duplicates(subset="link")


In [65]:
data_wikidata

,link,name,sitelinks,type
0,http://www.wikidata.org/entity/Q11036,the rolling stones,270,Group
1,http://www.wikidata.org/entity/Q1299,the beatles,237,Group
2,http://www.wikidata.org/entity/Q2331,led zeppelin,234,Group
3,http://www.wikidata.org/entity/Q11649,nirvana,226,Group
4,http://www.wikidata.org/entity/Q45188,coldplay,202,Group
...,...,...,...,...
19995,http://www.wikidata.org/entity/Q2545626,brett reed,7,Person
19996,http://www.wikidata.org/entity/Q2609394,branden steineckert,7,Person
19997,http://www.wikidata.org/entity/Q2642706,tomas asklund,7,Person
19998,http://www.wikidata.org/entity/Q7356419,rod de'ath,7,Person


In [49]:
data = pd.merge(data, data_wikidata, how="left", left_on="performer_single", right_on="name")


In [64]:
data.loc[data.name.isna(),"performer_single"].drop_duplicates().sample(60).head(30)


543                 leonard nimoy
72302                     fjellis
5017                          she
9480                    max roach
49730               fabulous four
60911                 pure orange
40687                guy mitchell
67441                roland white
41492               robert knight
56116    the swing easy orchestra
3864                   joe bataan
5330                 johnny adams
75820        the sound of feeling
1310                 steve marcus
23192                  joyce cobb
11501      billy "crash" craddock
62486                    monkey 3
11115              jesse crawford
24833               billy jackson
73487                   angeleyes
25152           sandra mihanovich
6858               ronnie ronalde
74733                junior brown
27153                      creamy
29157                ladi geisler
29547                elodie frege
7459           the crowe brothers
51798          crystal jacqueline
2019                    frankie j
47566         

In [5]:
data_humans1

,person,personLabel,sitelinks
0,http://www.wikidata.org/entity/Q254,Wolfgang Amadeus Mozart,518
1,http://www.wikidata.org/entity/Q2599,Paul McCartney,432
2,http://www.wikidata.org/entity/Q5383,David Bowie,390
3,http://www.wikidata.org/entity/Q23215,Adele,375
4,http://www.wikidata.org/entity/Q1203,John Lennon,354
...,...,...,...
9995,http://www.wikidata.org/entity/Q2545626,Brett Reed,7
9996,http://www.wikidata.org/entity/Q2609394,Branden Steineckert,7
9997,http://www.wikidata.org/entity/Q2642706,Tomas Asklund,7
9998,http://www.wikidata.org/entity/Q7356419,Rod de'Ath,7


In [24]:
import qwikidata


ModuleNotFoundError: No module named 'qwikidata'

In [2]:
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api


In [13]:
data_groups.musicalGroupLabel

0       The Rolling Stones
1              The Beatles
2             Led Zeppelin
3                  Nirvana
4                 Coldplay
               ...        
9995      Blankey Jet City
9996                  Echt
9997             Sun Caged
9998           Machine Men
9999               Van She
Name: musicalGroupLabel, Length: 10000, dtype: object

In [6]:
query_musicgroup("U2")


{'head': {'vars': ['group', 'groupLabel', 'sitelinks']},
 'results': {'bindings': [{'group': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q396'},
    'groupLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'U2'},
    'sitelinks': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '115'}}]}}